# Testing a Pre-Trained Network

Now that we know how to train an LSTM network, let's see try seeing the results on our own input text!

As we showed at the end of the Oriole notebook, we can load in a pretrained LSTM network using Tensorflow's Saver object. Before creating that object, we first have to create our Tensorflow graph. 

First, we declare some of our hyperparamters. 

In [1]:
numDimensions = 300
maxSeqLength = 550
batchSize = 24
lstmUnits = 64
numClasses = 2
#iterations = 100000
#iterations = 150000
#iterations = 200000
#iterations = 250000
iterations = 300000
#iterations = 350000

Then, we'll load in our data structures.

In [2]:
import numpy as np
with open('wordsList_glove_dim200.txt') as f:
    wordsList = [s.strip() for s in f.readlines()]
print('Loaded the word list!')
wordVectors = np.load('wordVectors_glove_dim200.npy')
print ('Loaded the word vectors!')

Loaded the word list!
Loaded the word vectors!


Next, we'll create our graph. This is the same code as we saw in the main Oriole notebook. 

In [3]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

Now, we load in the network. 

In [4]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
#saver.restore(sess, tf.train.latest_checkpoint('models3'))# 10
#saver.restore(sess, tf.train.latest_checkpoint('models4'))# 15
#saver.restore(sess, tf.train.latest_checkpoint('models5'))# 20
#saver.restore(sess, tf.train.latest_checkpoint('models7'))# 25
saver.restore(sess, tf.train.latest_checkpoint('models8'))# 30
#saver.restore(sess, tf.train.latest_checkpoint('models9))# 35

INFO:tensorflow:Restoring parameters from models8/pretrained_lstm.ckpt-300000


Before we input our own text, let's first define a couple of functions. The first is a function to make sure the sentence is in the proper format, and the second is a function that obtains the word vectors for each of the words in a given sentence. 

In [5]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = len(wordsList) - 1 #Vector for unkown words
    return sentenceMatrix

Now, we can create our input text. Try modifying the value for inputText, and see how the outputs change!

In [6]:
#1
f = open('test/NKE_p_test.txt', 'r')
fout = open('test_result_30/NKE_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [7]:
#1
f = open('test/NKE_n_test.txt', 'r')
fout = open('test_result_30/NKE_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [8]:
#2
f = open('test/MMM_p_test.txt', 'r')
fout = open('test_result_30/MMM_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [9]:
#2
f = open('test/MMM_n_test.txt', 'r')
fout = open('test_result_30/MMM_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [10]:
#3
f = open('test/AAPL_p_test.txt', 'r')
fout = open('test_result_30/AAPL_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [11]:
#3
f = open('test/AAPL_n_test.txt', 'r')
fout = open('test_result_30/AAPL_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [12]:
#4
f = open('test/JPM_p_test.txt', 'r')
fout = open('test_result_30/JPM_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [13]:
#4
f = open('test/JPM_n_test.txt', 'r')
fout = open('test_result_30/JPM_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [14]:
#5
f = open('test/GS_p_test.txt', 'r')
fout = open('test_result_30/GS_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [15]:
#5
f = open('test/GS_n_test.txt', 'r')
fout = open('test_result_30/GS_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [16]:
#6
f = open('test/GE_p_test.txt', 'r')
fout = open('test_result_30/GE_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [17]:
#6
f = open('test/GE_n_test.txt', 'r')
fout = open('test_result_30/GE_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [18]:
#7
f = open('test/MSFT_p_test.txt', 'r')
fout = open('test_result_30/MSFT_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [19]:
#7
f = open('test/MSFT_n_test.txt', 'r')
fout = open('test_result_30/MSFT_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [20]:
#8
f = open('test/XOM_p_test.txt', 'r')
fout = open('test_result_30/XOM_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [21]:
#8
f = open('test/XOM_n_test.txt', 'r')
fout = open('test_result_30/XOM_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [22]:
#9
f = open('test/BA_p_test.txt', 'r')
fout = open('test_result_30/BA_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [23]:
#9
f = open('test/BA_n_test.txt', 'r')
fout = open('test_result_30/BA_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [24]:
#10
f = open('test/WMT_p_test.txt', 'r')
fout = open('test_result_30/WMT_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [25]:
#10
f = open('test/WMT_n_test.txt', 'r')
fout = open('test_result_30/WMT_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [26]:
#11
f = open('test/INTC_p_test.txt', 'r')
fout = open('test_result_30/INTC_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [27]:
#11
f = open('test/INTC_n_test.txt', 'r')
fout = open('test_result_30/INTC_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [28]:
#12
f = open('test/PFE_p_test.txt', 'r')
fout = open('test_result_30/PFE_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [29]:
#12
f = open('test/PFE_n_test.txt', 'r')
fout = open('test_result_30/PFE_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [30]:
#13
f = open('test/CVX_p_test.txt', 'r')
fout = open('test_result_30/CVX_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [31]:
#13
f = open('test/CVX_n_test.txt', 'r')
fout = open('test_result_30/CVX_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [32]:
#14
f = open('test/IBM_p_test.txt', 'r')
fout = open('test_result_30/IBM_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [33]:
#14
f = open('test/IBM_n_test.txt', 'r')
fout = open('test_result_30/IBM_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [34]:
#15
f = open('test/MRK_p_test.txt', 'r')
fout = open('test_result_30/MRK_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [35]:
#15
f = open('test/MRK_n_test.txt', 'r')
fout = open('test_result_30/MRK_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [36]:
#16
f = open('test/VZ_p_test.txt', 'r')
fout = open('test_result_30/VZ_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [37]:
#16
f = open('test/VZ_n_test.txt', 'r')
fout = open('test_result_30/VZ_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [38]:
#17
f = open('test/JNJ_p_test.txt', 'r')
fout = open('test_result_30/JNJ_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [39]:
#17
f = open('test/JNJ_n_test.txt', 'r')
fout = open('test_result_30/JNJ_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [40]:
#18
f = open('test/CSCO_p_test.txt', 'r')
fout = open('test_result_30/CSCO_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [41]:
#18
f = open('test/CSCO_n_test.txt', 'r')
fout = open('test_result_30/CSCO_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [42]:
#19
f = open('test/CAT_p_test.txt', 'r')
fout = open('test_result_30/CAT_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [43]:
#19
f = open('test/CAT_n_test.txt', 'r')
fout = open('test_result_30/CAT_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [44]:
#20*
f = open('test/DIS_p_test.txt', 'r')
fout = open('test_result_30/DIS_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [45]:
#20*
f = open('test/DIS_n_test.txt', 'r')
fout = open('test_result_30/DIS_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [46]:
#21
f = open('test/PG_p_test.txt', 'r')
fout = open('test_result_30/PG_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [47]:
#21
f = open('test/PG_n_test.txt', 'r')
fout = open('test_result_30/PG_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [48]:
#22
f = open('test/KO_p_test.txt', 'r')
fout = open('test_result_30/KO_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [49]:
#22
f = open('test/KO_n_test.txt', 'r')
fout = open('test_result_30/KO_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [50]:
#23
f = open('test/MCD_p_test.txt', 'r')
fout = open('test_result_30/MCD_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [51]:
#23
f = open('test/MCD_n_test.txt', 'r')
fout = open('test_result_30/MCD_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [52]:
#24
f = open('test/HD_p_test.txt', 'r')
fout = open('test_result_30/HD_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [53]:
#24
f = open('test/HD_n_test.txt', 'r')
fout = open('test_result_30/HD_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [54]:
#25
f = open('test/AXP_p_test.txt', 'r')
fout = open('test_result_30/AXP_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [55]:
#25
f = open('test/AXP_n_test.txt', 'r')
fout = open('test_result_30/AXP_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [56]:
#26
f = open('test/UTX_p_test.txt', 'r')
fout = open('test_result_30/UTX_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [57]:
#26
f = open('test/UTX_n_test.txt', 'r')
fout = open('test_result_30/UTX_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [58]:
#27
f = open('test/UNH_p_test.txt', 'r')
fout = open('test_result_30/UNH_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [59]:
#27
f = open('test/UNH_n_test.txt', 'r')
fout = open('test_result_30/UNH_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [60]:
#28
f = open('test/V_p_test.txt', 'r')
fout = open('test_result_30/V_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [61]:
#28
f = open('test/V_n_test.txt', 'r')
fout = open('test_result_30/V_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [62]:
#29
f = open('test/TRV_p_test.txt', 'r')
fout = open('test_result_30/TRV_p_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()

In [63]:
#29
f = open('test/TRV_n_test.txt', 'r')
fout = open('test_result_30/TRV_n_test_result.txt', 'w')
for line in f:
    inputText = line
    inputMatrix = getSentenceMatrix(inputText)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        fout.write('1.0\t')
        fout.write(line)
    else:
        fout.write('-1.0\t')
        fout.write(line)
fout.close()